In [1]:
# Proyecto Final de Bootcamp SQL
# SucursalCF - SQLite
# Sebastián Tutistar Valencia
# Código Facilito

import sqlite3
import random
!pip install faker
from faker import Faker
from datetime import datetime, timedelta
import pandas as pd
import json

# Crear base en memoria
fake = Faker('es_ES')
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Crear tablas
cursor.executescript("""
CREATE TABLE Ubicaciones (
    IDUbicacion INTEGER PRIMARY KEY,
    Ciudad TEXT,
    Pais TEXT
);
CREATE TABLE Vendedores (
    IDVendedor INTEGER PRIMARY KEY,
    Vendedor TEXT
);
CREATE TABLE Oficinas (
    IDOficina INTEGER PRIMARY KEY,
    Oficina TEXT
);
CREATE TABLE Canales (
    IDCanal INTEGER PRIMARY KEY,
    Canal TEXT
);
CREATE TABLE Clientes (
    IDCliente INTEGER PRIMARY KEY,
    Cliente TEXT,
    IDUbicacion INTEGER,
    Segmento TEXT,
    Telefono TEXT,
    InfoAdicional TEXT,
    FOREIGN KEY (IDUbicacion) REFERENCES Ubicaciones(IDUbicacion)
);
CREATE TABLE Facturas (
    IDFactura INTEGER PRIMARY KEY,
    FechaFactura TEXT,
    IDCanal INTEGER,
    IDCliente INTEGER,
    IDVendedor INTEGER,
    Impuestos REAL,
    MontoFactura REAL,
    MontoSinImp REAL,
    Detalles TEXT,
    FOREIGN KEY (IDCanal) REFERENCES Canales(IDCanal),
    FOREIGN KEY (IDCliente) REFERENCES Clientes(IDCliente),
    FOREIGN KEY (IDVendedor) REFERENCES Vendedores(IDVendedor)
);
CREATE TABLE Cobranzas (
    IDCohanza INTEGER PRIMARY KEY,
    FechaCobranza TEXT,
    IDCliente INTEGER,
    IDFactura INTEGER,
    MontoCobranza REAL,
    FOREIGN KEY (IDCliente) REFERENCES Clientes(IDCliente),
    FOREIGN KEY (IDFactura) REFERENCES Facturas(IDFactura)
);
CREATE TABLE Proyectos (
    IDProyecto INTEGER PRIMARY KEY,
    CostoProyecto REAL,
    FechaFinProyecto TEXT,
    FechaInicioProyecto TEXT,
    IDCliente INTEGER,
    IDOficina INTEGER,
    Proyecto TEXT,
    FOREIGN KEY (IDCliente) REFERENCES Clientes(IDCliente),
    FOREIGN KEY (IDOficina) REFERENCES Oficinas(IDOficina)
);
""")

# Insertar datos aleatorios
paises = ['España', 'Francia', 'Alemania', 'Italia', 'Portugal', 'México', 'Argentina', 'Colombia', 'EE.UU.', 'Reino Unido']
for i in range(1, 101):
    cursor.execute("INSERT INTO Ubicaciones VALUES (?, ?, ?)", (i, fake.city(), random.choice(paises)))

for i in range(1, 101):
    cursor.execute("INSERT INTO Vendedores VALUES (?, ?)", (i, fake.name()))

tipos_oficina = ['Central', 'Norte', 'Sur', 'Este', 'Oeste', 'Regional', 'Internacional', 'Sucursal', 'Satélite']
for i in range(1, 101):
    cursor.execute("INSERT INTO Oficinas VALUES (?, ?)", (i, f"Oficina {random.choice(tipos_oficina)} {fake.random_number(digits=2)}"))

canales = ['Online', 'Tienda física', 'Telemarketing', 'Distribuidor', 'Mayorista', 'Partner', 'Marketplace', 'Redes Sociales', 'WhatsApp', 'Email']
for i in range(1, 101):
    cursor.execute("INSERT INTO Canales VALUES (?, ?)", (i, random.choice(canales)))

segmentos = ['Empresas', 'Minorista', 'Transporte', 'Exportador', 'Gobierno', 'Educación']
for i in range(1, 101):
    cursor.execute("INSERT INTO Clientes VALUES (?, ?, ?, ?, ?, ?)",
                  (i, fake.company(), random.randint(1, 100), random.choice(segmentos), fake.phone_number(),
                   json.dumps({"correo": fake.company_email(), "NIT": fake.random_number(digits=10)})))

for i in range(1, 101):
    monto_sin = round(random.uniform(100, 10000), 2)
    impuestos = round(monto_sin * 0.21, 2)
    monto_total = round(monto_sin + impuestos, 2)
    cursor.execute("INSERT INTO Facturas VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                  (i, fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d'),
                   random.randint(1, 100), random.randint(1, 100), random.randint(1, 100),
                   impuestos, monto_total, monto_sin,
                   json.dumps({"metodo_pago": random.choice(["Tarjeta", "Transferencia", "Efectivo"]),
                               "observaciones": fake.sentence(nb_words=6)})))

for i in range(1, 101):
    cursor.execute("INSERT INTO Cobranzas VALUES (?, ?, ?, ?, ?)",
                  (i, fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d'),
                   random.randint(1, 100), random.randint(1, 100),
                   round(random.uniform(100, 10000), 2)))

tipos_proyecto = ['Sistema de Gestión', 'Implementación ERP', 'Tienda Online', 'Migración Cloud',
                 'Rediseño Web', 'App Móvil', 'IA para Ventas', 'Portal Clientes', 'E-commerce',
                 'Automatización', 'Big Data', 'Seguridad Informática']
for i in range(1, 101):
    inicio = fake.date_between(start_date='-1y', end_date='today')
    fin = inicio + timedelta(days=random.randint(30, 365))
    cursor.execute("INSERT INTO Proyectos VALUES (?, ?, ?, ?, ?, ?, ?)",
                  (i, round(random.uniform(5000, 50000), 2),
                   fin.strftime('%Y-%m-%d'), inicio.strftime('%Y-%m-%d'),
                   random.randint(1, 100), random.randint(1, 100),
                   f"{random.choice(tipos_proyecto)} para {fake.word().capitalize()}"))

conn.commit()

# Ejecutar consultas
def run_query(sql):
    return pd.read_sql_query(sql, conn)

# 1. Cliente con mayor monto facturado
print("1. Cliente que ha generado el mayor monto de facturas:")
display(run_query("""
SELECT C.Cliente, SUM(F.MontoFactura) AS TotalFacturado
FROM Facturas F
JOIN Clientes C ON F.IDCliente = C.IDCliente
GROUP BY F.IDCliente
ORDER BY TotalFacturado DESC
LIMIT 1;
"""))

# 2. Vendedor con más ventas
print("2. Vendedor que ha generado más ventas:")
display(run_query("""
SELECT V.Vendedor, SUM(F.MontoFactura) AS TotalVentas
FROM Facturas F
JOIN Vendedores V ON F.IDVendedor = V.IDVendedor
GROUP BY F.IDVendedor
ORDER BY TotalVentas DESC
LIMIT 1;
"""))

# 3. Proyectos y costos por oficina
print("3. Cantidad de proyectos y costo total por oficina:")
display(run_query("""
SELECT O.Oficina, COUNT(P.IDProyecto) AS CantidadProyectos, SUM(P.CostoProyecto) AS CostoTotal
FROM Proyectos P
JOIN Oficinas O ON P.IDOficina = O.IDOficina
GROUP BY P.IDOficina
ORDER BY CantidadProyectos DESC;
"""))

# 4. Monto total de impuestos facturado por canal de venta
print("4. Monto total de impuestos facturado por canal de venta:")
display(run_query("""
SELECT C.Canal, SUM(F.Impuestos) AS TotalImpuestos
FROM Facturas F
JOIN Canales C ON F.IDCanal = C.IDCanal
GROUP BY F.IDCanal
ORDER BY TotalImpuestos DESC;
"""))

# 5. Ciudades con más clientes registrados
print("5. Ciudades con más clientes registrados:")
display(run_query("""
SELECT U.Ciudad, COUNT(C.IDCliente) AS TotalClientes
FROM Clientes C
JOIN Ubicaciones U ON C.IDUbicacion = U.IDUbicacion
GROUP BY C.IDUbicacion
ORDER BY TotalClientes DESC;
"""))

# 6. Proyectos con mayor duración
print("6. Proyectos con mayor duración:")
display(run_query("""
SELECT Proyecto, FechaInicioProyecto, FechaFinProyecto,
       julianday(FechaFinProyecto) - julianday(FechaInicioProyecto) AS DuracionDias
FROM Proyectos
ORDER BY DuracionDias DESC
LIMIT 5;
"""))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.1 MB/s eta 0:00:00
1. Cliente que ha generado el mayor monto de facturas:


,Cliente,TotalFacturado
0,Berenguer y Cortina S.A.,28186.21


2. Vendedor que ha generado más ventas:


,Vendedor,TotalVentas
0,Purificación Seguí,34595.88


3. Cantidad de proyectos y costo total por oficina:


...

,Canal,TotalImpuestos
0,Mayorista,4602.86
1,Marketplace,4483.06
2,Mayorista,3954.00
3,Tienda física,3936.66
4,Marketplace,3755.85
...,...,...
57,Marketplace,145.33
58,Marketplace,134.74
59,Partner,114.61
60,Email,76.31


5. Ciudades con más clientes registrados:


,Ciudad,TotalClientes
0,Sevilla,5
1,Guipúzcoa,3
2,Ávila,3
3,Asturias,3
4,Jaén,3
...,...,...
61,Baleares,1
62,Málaga,1
63,Ciudad,1
64,Lugo,1


6. Proyectos con mayor duración:


,Proyecto,FechaInicioProyecto,FechaFinProyecto,DuracionDias
0,Migración Cloud para Illo,2024-07-24,2025-07-23,364.0
1,Portal Clientes para Quos,2025-05-23,2026-05-17,359.0
2,Tienda Online para Molestias,2025-01-30,2026-01-19,354.0
3,App Móvil para Libero,2024-11-25,2025-11-13,353.0
4,Portal Clientes para Tempora,2025-04-02,2026-03-19,351.0
